In [7]:
from pyspark.sql import SparkSession

from pyspark.sql.functions import min, max, when, col, lit, udf
from pyspark.sql.types import DoubleType
from pyspark.ml.feature import MinMaxScaler, VectorAssembler
from pyspark.ml.linalg import VectorUDT

import pyspark
import dotenv
import os

In [5]:
pyspark.__file__

'/Users/lenn/anaconda3/envs/wow/lib/python3.11/site-packages/pyspark/__init__.py'

In [ ]:
env_file = 'creds.sh'
dotenv.load_dotenv(env_file, override=True)

CLIENT_ID = os.environ['CLIENT_ID']
SECRET_TOKEN = os.environ['SECRET_TOKEN']
USERNAME = os.environ['USERNAME']
PASSWORD = os.environ['PASSWORD']
USER_AGENT = 'MyBot/0.0.1'
USERNAME